<h1>E-commerce via Computer Vision</h1>

## Machine Learning: Addendum to 3rd Level of Fine Tuning (4th Step)

# At deployment time (March 2019)  the below-noted changes were made

Steps 1 through 4 were done exactly as show in the preceding notebooks, with two notable exceptions.   First, all notebooks were run with a Python 2 (2.7) kernel, rather than Python 3 (3.6), as doing so integrated well with GCP's native MLE query/prediction request service, discovery.   Second, rather than using VGG16's bottleneck layers as one object/1st layer of a concise model,  VGG16's model was simply reiterated in its full form and then used for bottlenecks and later sequential fine tuning.

## Once the ultimate fine tuning was run as before, deployment was effectuated as below (with these additional cells in that same Step4 Notebook):

In [ ]:
# Freeze the Keras model, save its session, make it a GraphDef, & elim. unnecessary indexing of 0 (only 1 img/MLE online predict)
K.set_learning_phase(0)  # testing/frozen
sess = K.get_session()
# Make GraphDef of Train Model
graph_train = sess.graph
graph_train_def = graph_util.convert_variables_to_constants(sess, graph_train.as_graph_def(), [my_model.output.name.replace(':0','')])


In [ ]:
# Build a 2nd graph to inject a bitstring-to-input_tensor pipeline/input pre-processor
with tf.Graph().as_default() as graph_input:
  # Define a function: map each bitstring ('input_bytes') to an RGB image ('input_tensor')
  # https://stackoverflow.com/questions/55008988/tensorflow-serving
  # -for-images-as-base64-encoded-strings-on-cloud-ml-engine
  def bytes_to_image(input_b64):
    image = tf.image.decode_image(input_b64, channels=3) # image in RGB
    image.set_shape([None, None, None])
    image = tf.image.resize_images(image, [224, 224])
    image = tf.divide(image, 255) # [0,1] scale for each pixel vector element
    image.set_shape([224, 224, 3])
    image = tf.reverse(image, axis=[2]) # RGB_2_BGR
    
    return image
    
  input_bytes = tf.placeholder(tf.string, shape=None, name="input_bytes")
  input_tensor = tf.map_fn(bytes_to_image, input_bytes, 
       back_prop=False, dtype=tf.float32)
  output = tf.identity(input_tensor, name='input_image')
  # Convert graph_input to a GraphDef
  graph_input_def = graph_input.as_graph_def()

In [ ]:
# Set the output directory, where the combined graph will be persisted
export_path = os.path.join(
      tf.compat.as_bytes(export_path_base),
      tf.compat.as_bytes(str(model_version_no)))

In [ ]:
# Build a 3rd graph, combining the prior 2, and then persist the combined graph as a SavedModel
with tf.Graph().as_default() as graph_combined:
    # https://github.com/hayatoy/cloudmlmagic/
    # blob/master/examples/Keras_Fine_Tuning.ipynb
    x = tf.placeholder(tf.string, name="input_b64")

    img, = tf.import_graph_def(graph_input_def,
                              input_map={'input_bytes:0': x},
                              return_elements=["input_image:0"])

    preds, = tf.import_graph_def(graph_train_def,
                                input_map={my_model.input.name: img},
                                return_elements=[my_model.output.name])
                                           
    # Now persist the final, combined graph to export_path
    with tf.Session() as sess2: 
        # inputs' alias suffix = '_bytes' for Cloud MLE
        inputs = {"image_bytes": tf.saved_model.utils.build_tensor_info(x)} 
        outputs = {"predictions":
           tf.saved_model.utils.build_tensor_info(preds)}
        signature = tf.saved_model.signature_def_utils.build_signature_def(
           inputs=inputs,
           outputs=outputs,
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME       
        )

        # save as SavedModel
        builder = tf.saved_model.builder.SavedModelBuilder(export_path)
        builder.add_meta_graph_and_variables(sess2,
            [tf.saved_model.tag_constants.SERVING],
                  signature_def_map={'serving_default': signature})
        builder.save()